In [2]:
import numpy as np
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer

In [3]:
text_data = np.array(['I love Brazil. Brazil!',
                     'Brazil is best',
                     'Germany beats both'])

In [4]:
count = CountVectorizer()
bag_of_words = count.fit_transform(text_data) # 단어의 등장 횟수 세기

features = bag_of_words.toarray()

target = np.array([0,0,1])

classifier = MultinomialNB(class_prior = [0.25, 0.5])

model = classifier.fit(features, target)

In [5]:
bag_of_words

<3x7 sparse matrix of type '<class 'numpy.int64'>'
	with 8 stored elements in Compressed Sparse Row format>

In [6]:
features

array([[0, 0, 0, 2, 0, 0, 1],
       [0, 1, 0, 1, 0, 1, 0],
       [1, 0, 1, 0, 1, 0, 0]])

In [7]:
count.get_feature_names()

['beats', 'best', 'both', 'brazil', 'germany', 'is', 'love']

# 18.3 이진 특성으로 나이브 베이즈 분류기 훈련하기

* 이진 특성이 나타나는 원핫인코딩 범주형 특성, 텍스트 분류 등에 많이 사용됨
* 평탄화 매개변수 alpha를 가지고 있음
* 사전 확률을 지정하려면 class_prior매개변수에 클래스별 사전확률을 담은 리스트를 전달
* 균등 분포를 사용하려면 fit_prior = False 사용

In [9]:
from sklearn.naive_bayes import BernoulliNB

features = np.random.randint(2, size = (100,3))
target = np.random.randint(2, size = (100,1)).ravel()

NumPy의 random 서브패키지에는 난수를 생성하는 다양한 명령을 제공한다. 그 중 가장 간단하고 많이 사용되는 것은 다음 3가지 명령이다.

* rand: 0부터 1사이의 균일 분포
* randn: 가우시안 표준 정규 분포
* randint: 균일 분포의 정수 난수
    * numpy.random.randint(low, high=None, size=None)
    * 만약 high를 입력하지 않으면 0과 low사이의 숫자를, high를 입력하면 low와 high는 사이의 숫자를 출력한다. size는 난수의 숫자이다.
    
<img src="files/스크린샷 2020-06-27 오후 12.02.41.png">

In [11]:

classifier = BernoulliNB(class_prior = [0.25, 0.5])
model = classifier.fit(features, target)


In [13]:
model_uniform_prior = BernoulliNB(class_prior = None, fit_prior = True)

# 18.4 예측 확률 보정하기
### Calibration
Calibration 이란 모형의 출력값이 실제 confidence (또는 이논문에서 calibrated confidence 로 표현) 를 반영하도록 만드는 것입니다. 예를 들어, X 의 Y1 에 대한 모형의 출력이 0.8이 나왔을 때, 80 % 확률로 Y1 일 것라는 의미를 갖도록 만드는 것입니다.

출처: https://3months.tistory.com/490 [Deep Play]
* 실제 confidence, accuracy 를 반영해야 한다

### method
‘sigmoid’ or ‘isotonic’ : The method to use for calibration. Can be ‘sigmoid’ which corresponds to Platt’s method (i.e. a logistic regression model) or ‘isotonic’ which is a non-parametric approach. It is not advised to use isotonic calibration with too few calibration samples (<<1000) since it tends to overfit.
* isotonic : 등위회귀. 비모수모델이므로 샘플 크기가 작으면 과대적합
* https://github.com/lseuny/sandbox/blob/master/calibrating_model_prediction.ipynb

In [14]:
from sklearn import datasets
from sklearn.naive_bayes import GaussianNB
from sklearn.calibration import CalibratedClassifierCV

In [15]:
iris = datasets.load_iris()
features = iris.data
target = iris.target

classifier = GaussianNB()

classifier_sigmoid = CalibratedClassifierCV(classifier, cv = 2, method = 'sigmoid')

classifier_sigmoid.fit(features, target)

new_observation = [[2.6, 2.6, 2.6, .4]]

classifier_sigmoid.predict_proba(new_observation)

array([[0.31859969, 0.63663466, 0.04476565]])

## predict proba 메서드
* 클래스 소속 확률
    * 모델이 해당 클래스를 예측할 확률이 90% 이상일 때만 사용할 수 있음
* 나이브 베이즈는 타깃 클래스에 대한 예측 확률의 순위는 유효하지만, 예측 확률이 0 또는 1에 극단적으로 가까워지는 경향이 있음
* CalibratedClassifierCV : k-fold 교차검증으로 예측 확률 클래스를 잘 보정해줘야 함
    * 반환된 예측 확률은 k-fold의 평균
    

In [16]:
classifier.fit(features, target).predict_proba(new_observation)

array([[2.31548432e-04, 9.99768128e-01, 3.23532277e-07]])

In [17]:
classifier_sigmoid.predict_proba(new_observation)

array([[0.31859969, 0.63663466, 0.04476565]])